In [118]:
# ------- SERVER EXTENSIONS ---------
lib =  r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities/droplet_dataset'
lib2 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities'
lib3 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/data_analysis'
lib4 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy'
lib5 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/scripts'
import sys
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)
import numpy as np
import pandas as pd
import scipy
from os.path import join
import sklearn
from sklearn.manifold import TSNE
import pickle
# from Bio.Cluster import kcluster
import os
import numpy as np
import yaml
import os
import pandas
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sys
import seaborn as sns
# import pyclustering
import matplotlib.pylab as plt
import seaborn as sb
from scipy.spatial.distance import cdist

from shutil import copyfile
import matplotlib.pyplot as plt

from utilities.general_helpers import flatten_list
import os
from os.path import join
import sklearn
from droplet_dataset import *
from utilities import *
from matplotlib import pyplot
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import random
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
sklearn.__version__
%matplotlib notebook
# ------- SERVER EXTENSIONS ---------

###### build markers table

In [75]:
MARKERS_PATH = r'/storage/md_keren/shitay/Data/tables/ImmuneCellsMarkersUpdated_12.11.20.xlsx'

xls = pd.ExcelFile(MARKERS_PATH)
positive_markers_df = pd.read_excel(xls, 'and_or')
negative_markers_df = pd.read_excel(xls, 'none')

from classifying_cell_types import *
positive_markers_table = builds_cell_type_markers_table(positive_markers_df)
negative_markers_table = builds_cell_type_markers_table(negative_markers_df)


cell_types_markers_mapping = {k: [vv.split(';') for vv in v] for k,v in positive_markers_table.items()}
cell_types_markers_mapping = {k: flatten_list(v) for k,v in cell_types_markers_mapping.items()}
markers_cell_types_mapping = {}
for k, v in ff.items():
    for marker in v:
        if marker in markers_cell_types_mapping.keys():
            markers_cell_types_mapping[marker].append(k)
        else:
            markers_cell_types_mapping[marker] = [k]

REDUCED_MHC2_GENES = ['HLA-DMA',  'HLA-DMB',  'HLA-DOA',  'HLA-DOB',
                      'HLA-DPA1',  'HLA-DPB1',  'HLA-DQA1',  'HLA-DQB1',
                      'HLA-DQB2',  'HLA-DRA',  'HLA-DRB1',  'HLA-DRB5']
for MHCII_gene in REDUCED_MHC2_GENES:
    markers_cell_types_mapping[MHCII_gene] = markers_cell_types_mapping['MHCII']
    
markers_cell_types_mapping.pop('MHCII' ,None);

In [186]:
markers_cell_types_mapping.keys()

dict_keys(['CD3E', 'CD4', 'CD8A', 'CD8B', 'FOXP3', 'CTLA4', 'IL2RA', 'NCR1', 'NCAM1', 'FCGR3A', 'CD19', 'MS4A1', 'CD2', 'CD28', 'CD69', 'ICOS', 'TNFRSF4', 'TNFRSF9', 'CD27', 'IL2', 'TNF', 'IFNG', 'KLRG1', 'B3GAT1', 'HAVCR2', 'PDCD1', 'LAG3', 'BTLA', 'CDKN2A', 'CDKN1A', 'TP53', 'FASLG', 'EOMES', 'ID2', 'KIR3DL1', 'ADORA2A', 'HAVCR1', 'CCR7', 'ID3', 'TCF7', 'HNF1A', 'BCL2L11', 'SELL', 'CD163', 'ITGAM', 'HLA-DRA', 'CD14', 'FCGR1A', 'ITGAX', 'THBD', 'IL3RA', 'CLEC4C', 'NRP1', 'LILRA4', 'CD33', 'FUT4', 'FCGR3B', 'CXCR1', 'CXCR2', 'G0S2', 'CSF3R', 'CEACAM1', 'CEACAM6', 'CEACAM3', 'ENPP3', 'PTPRC', 'CD1C', 'HLA-DMA', 'HLA-DMB', 'HLA-DOA', 'HLA-DOB', 'HLA-DPA1', 'HLA-DPB1', 'HLA-DQA1', 'HLA-DQB1', 'HLA-DQB2', 'HLA-DRB1', 'HLA-DRB5'])

##### Build Dataframe

In [182]:
def transpose_list(l):
    return list(map(list, zip(*l)))

df = pd.DataFrame(transpose_list([markers_cell_types_mapping.keys(), markers_cell_types_mapping.values(), 
                                  [0]*len(markers_cell_types_mapping),
                                  *[[0]*len(markers_cell_types_mapping) for _ in range(len(samples))]]),
                                  columns=['Marker', 'Cell_types', 'Count', *sorted(samples)])




#### go over all samples

In [183]:
THRESHOLD = 1 # min value for a gene expression to be counted


ROW_SAMPLES_PATH = fr'/storage/md_keren/shitay/Data/droplet_seq/ROW_DATA'
SAMPLES_INFORMATION_PATH = fr'/storage/md_keren/shitay/Data/inferCNV_data/update_runs/24.5.21'
MARKERS_PATH = r'/storage/md_keren/shitay/Data/tables/ImmuneCellsMarkersUpdated_12.11.20.xlsx'


samples = [subfolder for subfolder in os.listdir(ROW_SAMPLES_PATH) if subfolder.startswith('M') and (not subfolder in not_investigate)]
number_of_cells = 0
from os.path import join
for sample_id in sorted(samples):
    print(sample_id)
    rna_sample = loading_sample(row_data_path=join(ROW_SAMPLES_PATH, fr'{sample_id}'),
                                    cells_information_path=join(SAMPLES_INFORMATION_PATH, fr'{sample_id}'))
    conflict_rna_sample = rna_sample.filter_cells_by_property('cancer_immune_conflict', True)
    conflict_rna_sample.normalize_data()
    number_of_cells += rna_sample.number_of_cells
    for marker, cell_Types in markers_cell_types_mapping.items():
        if not marker in conflict_rna_sample.gene_names:
            print(f'{marker} is not shown in list')
            continue
        count = sum(conflict_rna_sample.counts[:, conflict_rna_sample.gene_names.index(marker)] > THRESHOLD)
        df.loc[df['Marker']==marker, 'Count'] += count
        df.loc[df['Marker']==marker, sample_id] = ', '.join([str(rna_sample.number_of_cells), str(count)])

print(f'Number of cells: {number_of_cells}')
df

M100.pkl
IL2 is not shown in list
B3GAT1 is not shown in list
HAVCR1 is not shown in list
CEACAM6 is not shown in list
CEACAM3 is not shown in list
CD1C is not shown in list
M101.pkl
HAVCR1 is not shown in list
HNF1A is not shown in list
CEACAM6 is not shown in list
M102.pkl
HNF1A is not shown in list
CEACAM6 is not shown in list
M103.pkl
NCR1 is not shown in list
CD19 is not shown in list
HAVCR1 is not shown in list
HNF1A is not shown in list
M104.pkl
IL2 is not shown in list
CLEC4C is not shown in list
LILRA4 is not shown in list
CXCR1 is not shown in list
CXCR2 is not shown in list
CEACAM6 is not shown in list
CEACAM3 is not shown in list
CD1C is not shown in list
M105.pkl
M106.pkl
CEACAM6 is not shown in list
M107.pkl
HNF1A is not shown in list
CEACAM6 is not shown in list
M108.pkl
CTLA4 is not shown in list
TNFRSF9 is not shown in list
IL2 is not shown in list
IFNG is not shown in list
PDCD1 is not shown in list
KIR3DL1 is not shown in list
HAVCR1 is not shown in list
CLEC4C is no

,Marker,Cell_types,Count,M100.pkl,M101.pkl,M102.pkl,M103.pkl,M104.pkl,M105.pkl,M106.pkl,...,M139.pkl,M140.pkl,M141.pkl,M143.pkl,M144.pkl,M145.pkl,M146.pkl,M97.pkl,M98.pkl,M99.pkl
0,CD3E,"[T cells, CD4 helper T cells, CD8 Cytotoxic T ...",5732,"4055, 17","5141, 8","22694, 783","2453, 1","4944, 1","14312, 875","5136, 45",...,"6260, 4","3756, 14","1233, 4","10992, 10","10765, 43","8911, 111","8281, 256","4296, 111","5137, 39","5093, 36"
1,CD4,"[CD4 helper T cells, Regulatory CD4 T cells, R...",6506,"4055, 14","5141, 54","22694, 136","2453, 8","4944, 16","14312, 174","5136, 43",...,"6260, 4","3756, 10","1233, 1","10992, 134","10765, 41","8911, 21","8281, 158","4296, 129","5137, 47","5093, 228"
2,CD8A,"[CD8 Cytotoxic T cells, Regulatory CD8 T cells...",3017,"4055, 5","5141, 9","22694, 233","2453, 0","4944, 1","14312, 198","5136, 27",...,"6260, 1","3756, 2","1233, 1","10992, 4","10765, 25","8911, 82","8281, 203","4296, 91","5137, 3","5093, 23"
3,CD8B,"[CD8 Cytotoxic T cells, Regulatory CD8 T cells...",2861,"4055, 3","5141, 9","22694, 130","2453, 0","4944, 1","14312, 162","5136, 24",...,"6260, 1","3756, 4","1233, 2","10992, 1","10765, 31","8911, 82","8281, 188","4296, 84","5137, 10","5093, 17"
4,FOXP3,"[Regulatory T cells, Regulatory CD4 T cells, R...",151,"4055, 0","5141, 0","22694, 4","2453, 0","4944, 1","14312, 10","5136, 6",...,"6260, 0","3756, 0","1233, 0","10992, 1","10765, 0","8911, 2","8281, 13","4296, 4","5137, 0","5093, 1"
5,CTLA4,"[Regulatory T cells, Regulatory CD4 T cells, R...",1093,"4055, 3","5141, 0","22694, 2","2453, 0","4944, 3","14312, 19","5136, 16",...,"6260, 1","3756, 0","1233, 0","10992, 1","10765, 8","8911, 8","8281, 90","4296, 27","5137, 1","5093, 4"
6,IL2RA,"[Regulatory T cells, Regulatory CD4 T cells, R...",673,"4055, 2","5141, 1","22694, 9","2453, 0","4944, 0","14312, 31","5136, 12",...,"6260, 0","3756, 2","1233, 0","10992, 39","10765, 4","8911, 2","8281, 21","4296, 22","5137, 9","5093, 21"
7,NCR1,"[NKT cells, NK cells, Immune_general]",160,"4055, 0","5141, 0","22694, 16",0,"4944, 0","14312, 7","5136, 0",...,"6260, 0","3756, 0","1233, 0","10992, 0","10765, 0","8911, 3","8281, 7","4296, 0",0,"5093, 1"
8,NCAM1,"[NKT cells, NK cells]",525,"4055, 1","5141, 2","22694, 3","2453, 0","4944, 36","14312, 9","5136, 22",...,"6260, 0","3756, 0","1233, 2","10992, 123","10765, 6","8911, 58","8281, 11","4296, 2","5137, 0","5093, 6"
9,FCGR3A,"[NK cells, Neutrophils]",6283,"4055, 18","5141, 22","22694, 202","2453, 0","4944, 6","14312, 125","5136, 27",...,"6260, 4","3756, 2","1233, 1","10992, 38","10765, 51","8911, 26","8281, 198","4296, 197","5137, 54","5093, 192"


In [184]:
df.columns = [vv.replace('.pkl', '') for vv in list(df.columns)]
df.to_csv(fr'/storage/md_keren/shitay/garbage/tumor_cells_expressing_immune_markers_thre_{THRESHOLD}.csv')